In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pathlib import Path
import struct
import os

# Configuration
SIM_RES = 2500
LENSPLANE_DIR = Path('/mnt/home/mlee1/ceph/hydro_replace_lensplanes')
EXPECTED_GRID = 4096  # LP_GRID size

In [ ]:
# List available lens plane directories
print("Lens Plane Directories:")
print("=" * 70)

if not LENSPLANE_DIR.exists():
    print(f"Directory not found: {LENSPLANE_DIR}")
else:
    dirs = sorted([d for d in LENSPLANE_DIR.iterdir() if d.is_dir()])
    for d in dirs:
        # Count files
        bin_files = list(d.glob('snap*.bin'))
        config_exists = (d / 'config.dat').exists()
        print(f"{d.name}: {len(bin_files)} bin files, config.dat: {'✓' if config_exists else '✗'}")

In [ ]:
def read_density_file(filepath):
    """Read binary density file in lux PreProjected format.
    
    Format: int32(grid) + float64[N*N](delta*dz) + int32(grid)
    """
    with open(filepath, 'rb') as f:
        # Read header grid size
        header_grid = struct.unpack('i', f.read(4))[0]
        
        # Read density field
        n_elements = header_grid * header_grid
        data = np.frombuffer(f.read(8 * n_elements), dtype=np.float64)
        field = data.reshape((header_grid, header_grid))
        
        # Read footer grid size
        footer_grid = struct.unpack('i', f.read(4))[0]
        
    return {
        'header_grid': header_grid,
        'footer_grid': footer_grid,
        'field': field,
        'valid': header_grid == footer_grid
    }

def read_config_dat(filepath):
    """Read lux config.dat file."""
    config = {}
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if '=' in line and not line.startswith('#'):
                key, value = line.split('=', 1)
                config[key.strip()] = value.strip()
    return config

In [ ]:
# Select a model directory to analyze
MODEL = 'dmo'  # Change as needed: dmo, hydro, replace_Mgt12.5, bcm_arico20_Mgt12.5, etc.

model_dir = LENSPLANE_DIR / f'L205n{SIM_RES}TNG_{MODEL}'

if not model_dir.exists():
    print(f"Directory not found: {model_dir}")
    print(f"\nAvailable directories:")
    for d in sorted(LENSPLANE_DIR.iterdir()):
        print(f"  {d.name}")
else:
    print(f"Analyzing: {model_dir}")
    
    # List files
    bin_files = sorted(model_dir.glob('snap*.bin'))
    print(f"\nFound {len(bin_files)} density files")

In [ ]:
# Check config.dat
config_path = model_dir / 'config.dat'
if config_path.exists():
    config = read_config_dat(config_path)
    print("config.dat contents:")
    print("-" * 50)
    for key, value in config.items():
        print(f"  {key} = {value}")
else:
    print("config.dat not found!")

In [ ]:
# Validate all density files
print("\nDensity File Validation:")
print("-" * 70)
print(f"{'File':<25} {'Grid':<10} {'Valid':<8} {'Min':<15} {'Max':<15}")
print("-" * 70)

file_info = []
for f in bin_files[:10]:  # Check first 10
    try:
        data = read_density_file(f)
        valid = '✓' if data['valid'] and data['header_grid'] == EXPECTED_GRID else '✗'
        print(f"{f.name:<25} {data['header_grid']:<10} {valid:<8} {data['field'].min():<15.6e} {data['field'].max():<15.6e}")
        file_info.append(data)
    except Exception as e:
        print(f"{f.name:<25} ERROR: {e}")

if len(bin_files) > 10:
    print(f"... and {len(bin_files) - 10} more files")

In [ ]:
# Visual inspection of a density field
if bin_files:
    # Load a specific file
    SNAP_FILE = bin_files[0]  # Change to inspect different snapshots
    
    data = read_density_file(SNAP_FILE)
    field = data['field']
    
    print(f"File: {SNAP_FILE.name}")
    print(f"Grid: {data['header_grid']}")
    print(f"Min: {field.min():.6e}")
    print(f"Max: {field.max():.6e}")
    print(f"Mean: {field.mean():.6e}")
    print(f"Std: {field.std():.6e}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Full field
    ax = axes[0]
    # Add small offset to avoid log(0)
    field_plot = field.copy()
    field_plot[field_plot <= 0] = np.nan
    
    im = ax.imshow(field_plot, norm=LogNorm(vmin=np.nanpercentile(field_plot, 1), 
                                             vmax=np.nanpercentile(field_plot, 99)),
                   cmap='magma', origin='lower')
    ax.set_title(f'Full Field: {SNAP_FILE.name}')
    ax.set_xlabel('x [pixels]')
    ax.set_ylabel('y [pixels]')
    plt.colorbar(im, ax=ax, label=r'$\delta \cdot dz$')
    
    # Zoom to center
    ax = axes[1]
    center = field.shape[0] // 2
    half = 512
    region = field_plot[center-half:center+half, center-half:center+half]
    
    im = ax.imshow(region, norm=LogNorm(vmin=np.nanpercentile(field_plot, 1),
                                         vmax=np.nanpercentile(field_plot, 99)),
                   cmap='magma', origin='lower')
    ax.set_title('Center 1024x1024 pixels')
    ax.set_xlabel('x [pixels]')
    ax.set_ylabel('y [pixels]')
    plt.colorbar(im, ax=ax, label=r'$\delta \cdot dz$')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Pixel value distribution
if bin_files:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot distribution for a few snapshots
    for i, f in enumerate(bin_files[:5]):
        data = read_density_file(f)
        vals = data['field'].flatten()
        vals = vals[vals > 0]
        if len(vals) > 0:
            ax.hist(np.log10(vals), bins=100, alpha=0.5, label=f.stem, density=True)
    
    ax.set_xlabel(r'log10($\delta \cdot dz$)')
    ax.set_ylabel('Density')
    ax.set_title(f'Pixel Value Distribution - {MODEL}')
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare multiple models for same snapshot
SNAP_NUM = 99  # Pick a snapshot to compare

models_to_compare = ['dmo', 'hydro', 'replace_Mgt12.5', 'bcm_arico20_Mgt12.5']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

fields = {}
for i, model in enumerate(models_to_compare):
    model_dir = LENSPLANE_DIR / f'L205n{SIM_RES}TNG_{model}'
    snap_file = model_dir / f'snap{SNAP_NUM:03d}.bin'
    
    ax = axes[i]
    
    if snap_file.exists():
        data = read_density_file(snap_file)
        field = data['field']
        fields[model] = field
        
        field_plot = field.copy()
        field_plot[field_plot <= 0] = np.nan
        
        # Zoom to center
        center = field.shape[0] // 2
        half = 512
        region = field_plot[center-half:center+half, center-half:center+half]
        
        im = ax.imshow(region, norm=LogNorm(vmin=np.nanpercentile(field_plot, 1),
                                             vmax=np.nanpercentile(field_plot, 99)),
                       cmap='magma', origin='lower')
        ax.set_title(model)
        plt.colorbar(im, ax=ax)
    else:
        ax.text(0.5, 0.5, 'Not found', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{model} (missing)')

plt.suptitle(f'Lens Plane Comparison - Snap {SNAP_NUM}', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Summary
print("=" * 70)
print("LENS PLANE SUMMARY")
print("=" * 70)

for d in sorted(LENSPLANE_DIR.iterdir()):
    if d.is_dir():
        bin_files = list(d.glob('snap*.bin'))
        config_ok = (d / 'config.dat').exists()
        
        # Check a sample file
        grid_ok = False
        if bin_files:
            try:
                data = read_density_file(bin_files[0])
                grid_ok = data['valid'] and data['header_grid'] == EXPECTED_GRID
            except:
                pass
        
        status = '✓' if config_ok and grid_ok and len(bin_files) > 0 else '✗'
        print(f"{d.name:<45} {status} ({len(bin_files)} files)")